## Exploring Google Location History

In [ ]:
!pip install geopandas

In [ ]:
!pip install descartes

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gp
import shapely.geometry as sg
import datetime as dt
from matplotlib import cm
from matplotlib.lines import Line2D

In [ ]:
def parse_json(json):
    points = [sg.Point(xy) for xy in zip(json.locations.apply(lambda x: x["longitudeE7"] / 10000000),
                                     json.locations.apply(lambda x: x["latitudeE7"] / 10000000))]
    df = gp.GeoDataFrame(geometry=points)
    locations = json.locations
    df["timestamp"] = locations.apply(lambda x: pd.to_datetime(x["timestampMs"], unit='ms'))
    df["accuracy"] = locations.apply(lambda x: x["accuracy"])
    df["longitude"] = locations.apply(lambda x: x["longitudeE7"] / 10000000)
    df["latitude"] = locations.apply(lambda x: x["latitudeE7"] / 10000000)
    df["location"] = df['latitude'].astype(str).str.cat(df['longitude'].astype(str), sep =",")     
    return df

In [ ]:
def compute_legend(df, ax, cmap, steps):
    vals = [x / (steps - 1) for x in range(steps)]
    custom_lines = [Line2D([0], [0], color=cmap(step), lw=4) for step in vals]
    labels = [df.timestamp.quantile(step).strftime("%Y-%m-%d") for step in vals]
    ax.legend(custom_lines, labels, loc="lower right")

def seconds(timestamp):
    return timestamp.to_pydatetime().timestamp()

def calculate_pal(df2, cmap):
    min_ts = seconds(df2.timestamp.min())
    max_ts = seconds(df2.timestamp.max())
    true_min = max_ts - min_ts
    if true_min == 0:
        true_min = 1
    return df2.timestamp.apply(lambda ts: cmap((seconds(ts) - min_ts) / (true_min))).tolist()

def draw_map(df, box, region):
    box_only = df[df.geometry.within(box)]    
    if box_only.size > 0:
        minx, miny, maxx, maxy = box.bounds
        base = world.plot(color='white', edgecolor='silver', figsize=(16,12))
        base.set_xlim(minx, maxx)
        base.set_ylim(miny, maxy)
        cmap = cm.get_cmap('viridis')
        pal = calculate_pal(box_only, cmap)
        ax = box_only.plot(ax=base, marker='o', color=pal, markersize=8)
        compute_legend(box_only, ax, cmap, 5)
    else:
        print('No Data for ' + region)

### Load Data

In [ ]:
%time df = parse_json(pd.read_json("Location History.json"))
df.shape

In [ ]:
df.to_csv('location_data.csv')

## Data Stats

In [ ]:
df["accuracy"].describe()

In [ ]:
df["location"].describe()

In [ ]:
df["timestamp"].groupby(df["timestamp"].astype("datetime64").dt.month).count().plot(kind="bar")

### Download Map Data to Map History

Reference: Country Level Geos

https://gist.github.com/graydon/11198540

In [ ]:
# http://thematicmapping.org/downloads/world_borders.php
!wget http://thematicmapping.org/downloads/TM_WORLD_BORDERS-0.3.zip

In [ ]:
!unzip TM_WORLD_BORDERS-0.3.zip

In [ ]:
world = gp.read_file('TM_WORLD_BORDERS-0.3.shp')

## World Map

In [ ]:
draw_map(df, sg.box(-140, -20, 140, 70), 'World')

## USA

In [ ]:
draw_map(df, sg.box(-130, 21, -70, 55), 'USA') # minus Alaska & Hawaii

## Map of Europe

In [ ]:
draw_map(df, sg.box(-10, 30, 50, 70), 'Europe')

## Japan

In [ ]:
draw_map(df, sg.box(125, 25, 150, 50), 'Japan')

## Okinawa

In [ ]:
draw_map(df, sg.box(127.4, 26, 128.5, 27), 'Okinawa')